<h2> Load Data</h2>

In [1]:
import warnings

%matplotlib qt
import matplotlib.pyplot as plt
import pandas as pd
import preparers
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
data = preparers.load_queried(
    participant_number=1,
    snippets=["BogoSort", "Ackerman"],
    query_code=True,
    query_eeg=True,
    query_eye_tracking=True,
    query_behavioral=True,
    query_log=True,
    query_input=True,
    query_meta=True,
)

Loading Ackerman Files .........
Loading BogoSort Files .........


<h2>potential funtion arguments</h2>

In [3]:
fig, ax = (None, None)
image_path = "./screenshots/BogoSort.png"
display_width = 1920
display_height = 1080

<h2> make animation</h2>

In [4]:
df_eyetracking = data["BogoSort"]["Code"]["EyeTracking"][
    ["l_valid", "r_valid", "l_display_x", "l_display_y", "r_display_x", "r_display_y", "time", "l_pupil_diameter", "r_pupil_diameter",]
]
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].apply(lambda x: x * display_width)
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(int)

df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].apply(lambda x: x * display_width)
df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(int)

df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].apply(lambda x: x * display_height)
df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(int)

df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].apply(lambda x: x * display_height)
df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(int)

raw_eeg = data["BogoSort"]["Code"]["EEG"]

In [13]:
import math
import tkinter as tk

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.gridspec import GridSpec
from pandas import DataFrame


class DataStreamer:
    def __draw_image_to(self, axis, image, title=""):
        axis.imshow(image)
        axis.set_title(title)
        axis.axis("off")

    def __draw_plot_to(self, axis, x, y, current_time, style="c-", title="", xlabel="", ylabel=""):
        axis.plot(x, y, style)
        axis.set_title(title)
        axis.set_xlabel(xlabel)
        axis.set_ylabel(ylabel)
        axis.axvline(
            x=current_time, color="k", linestyle="--",
        )

    def __draw_eeg_to(self, axis, raw, time, title=""):
        axis.set_title(title)
        df_eeg = raw.to_data_frame()
        df_time = df_eeg["time"]
        df_eeg = df_eeg.drop(["x_dir", "y_dir", "z_dir"], axis=1)
        df_eeg = df_eeg.set_index("time")

        current = 0.0
        step = math.ceil(max(abs(min(df_eeg.min())), abs(max(df_eeg.max()))) / 10.0)
        y_ticks = []
        for name, value in df_eeg.iteritems():
            df_eeg[name] = df_eeg[name] + current
            y_ticks.append(current)
            current += step

        axis.plot(df_time, df_eeg)
        axis.set_xlim(0, df_time.max())
        axis.axvline(
            x=time, color="k", linestyle="--",
        )
        axis.set_yticks(y_ticks)
        axis.set_yticklabels(df_eeg.columns)

    # helper functions for matplotlib and gui setup

    def __plot_to_gui(self, figure, root, column, row, columnspan=1, rowspan=1, sticky="news"):
        tk_fig = FigureCanvasTkAgg(figure, root)
        tk_fig.get_tk_widget().grid(column=column, row=row, columnspan=columnspan, rowspan=rowspan, sticky="nw")

    def __create_figure(self, figsize, dpi=80):
        fig = plt.Figure(figsize=figsize, dpi=dpi)
        ax = fig.add_subplot(111)
        return fig, ax

    # gui building functions start here

    def __draw_main_image(self):
        (fig, ax) = self.__create_figure((self.main_image_width, self.main_image_height))
        self.__draw_image_to(ax, self.base_image, title="Main Image")
        width = self.left_eye_current_width
        self.__plot_to_gui(fig, root, width, 0, self.main_image_width, self.main_image_height)

    def __draw_left_eye_gaze(self):
        (fig, ax) = self.__create_figure((self.left_eye_gaze_width, self.left_eye_gaze_height))
        self.__draw_plot_to(
            ax,
            self.df_eyetracking["time"],
            self.df_eyetracking["l_pupil_diameter"],
            self.current_time,
            style="c-",
            title="left pupil diameter",
            xlabel="time in seconds",
            ylabel="diameter in mm",
        )
        self.__plot_to_gui(fig, root, 0, 0, self.left_eye_gaze_height, self.left_eye_gaze_width)

    def __draw_right_eye_gaze(self):
        (fig, ax) = self.__create_figure((self.right_eye_gaze_width, self.right_eye_gaze_height))
        self.__draw_plot_to(
            ax,
            self.df_eyetracking["time"],
            self.df_eyetracking["r_pupil_diameter"],
            self.current_time,
            style="c-",
            title="right pupil diameter",
            xlabel="time in seconds",
            ylabel="diameter in mm",
        )
        height = self.left_eye_gaze_height
        self.__plot_to_gui(fig, root, 0, height, self.right_eye_gaze_height, self.right_eye_gaze_width)

    def __draw_left_eye_current(self):
        (fig, ax) = self.__create_figure((self.left_eye_current_width, self.left_eye_current_height))
        # todo - change image
        self.__draw_image_to(ax, self.base_image, title="Current Left Eye")
        height = self.left_eye_gaze_height + self.right_eye_gaze_height
        self.__plot_to_gui(fig, root, 0, height, self.left_eye_current_height, self.left_eye_current_width)

    def __draw_right_eye_current(self):
        (fig, ax) = self.__create_figure((self.right_eye_current_width, self.right_eye_current_height))
        # todo - change image
        self.__draw_image_to(ax, self.base_image, title="Current Right Eye")
        height = self.left_eye_gaze_height + self.right_eye_gaze_height + self.left_eye_current_height
        self.__plot_to_gui(fig, root, 0, height, self.right_eye_current_height, self.right_eye_current_width)

    # init from here on

    def __init__(self, master, image_path, df_eyetracking, raw_eeg):
        self.master = master
        master.title("Eyetracking-EEG Analysis Tool")
        self.current_time = 3970

        self.base_image = plt.imread(image_path)

        self.main_image_width = 12
        self.main_image_height = 12

        self.left_eye_gaze_width = 4
        self.left_eye_gaze_height = 2

        self.right_eye_gaze_width = 4
        self.right_eye_gaze_height = 2

        self.left_eye_current_width = 4
        self.left_eye_current_height = 4

        self.right_eye_current_width = 4
        self.right_eye_current_height = 4

        self.top_plot_width = 4
        self.top_plot_height = 4

        self.eeg_channels_width = 8
        self.eeg_channels_height = 8

        self.df_eeg = raw_eeg.to_data_frame()
        self.df_eeg = self.df_eeg.drop(["x_dir", "y_dir", "z_dir"], axis=1)

        self.df_eyetracking = df_eyetracking

        self.__draw_left_eye_gaze()
        self.__draw_right_eye_gaze()
        self.__draw_left_eye_current()
        self.__draw_right_eye_current()
        self.__draw_main_image()


root = tk.Tk()
window = DataStreamer(root, image_path, df_eyetracking, raw_eeg)
root.mainloop()

In [12]:
import mne

something = mne.read_raw_eeg("./filteredData/Participant01/EEG/Code_BogoSort_raw.fif")

AttributeError: module 'mne' has no attribute 'read_raw_eeg'

In [8]:
raw_eeg.plot_sensors()

RuntimeError: No valid channel positions found